In [15]:
import pandas as pd
import numpy as np
import os

In [53]:
name = 'P100_ms_cb'
model_name = name
file_name = name

In [54]:
DATA_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/%s/raw_data" %name)
SAVE_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/%s/" %name)

## Dense Layer

In [4]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfDense

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [ ]:
dfDense.describe()

In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [67]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,activation_fct,use_bias,optimizer,gpu_count,timeUsed_median,timeUsed_min,timeUsed_max,timeUsed_std,runtime_median
0,64,177,5,17,45,3,0,32,0,False,5,2,0.316345,0.262983,0.390247,0.052181,0.336130
1,49,254,4,2,16,3,1,32,1,False,1,2,0.081119,0.076643,0.084578,0.003248,0.080812
2,11,402,5,13,5,3,1,32,0,False,5,2,0.184340,0.172904,0.204071,0.012873,0.181507
3,8,279,4,33,19,3,1,32,2,True,1,2,0.164374,0.151647,0.182422,0.012626,0.163857
4,55,233,1,30,7,1,1,32,3,True,4,2,0.991728,0.769319,1.017806,0.111501,0.979455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,34,442,5,15,2,4,1,32,2,False,2,2,0.770191,0.769399,0.781417,0.005488,0.778081
996,39,335,2,12,10,4,0,32,3,True,6,2,0.408066,0.402259,0.416446,0.005823,0.426931
997,64,96,6,37,96,4,1,32,2,True,6,2,0.162839,0.159523,0.170519,0.004605,0.178805
998,23,314,3,25,23,1,1,32,0,True,2,2,0.817402,0.789535,0.833787,0.018267,0.857706


In [68]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [69]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [59]:
print(dfConv.duplicated().sum())

0


In [70]:
dfConv.describe()

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu_count,...,opt_Adadelta,opt_Adagrad,opt_Adam,opt_Momentum,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.0,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,32.607600,256.190600,3.971800,63.529400,69.330200,2.474800,0.494400,32.0,0.506000,2.0,...,0.168400,0.163000,0.163200,0.167200,0.173000,0.165200,0.235000,0.248600,0.261400,0.255000
std,18.569862,147.667552,1.988818,320.365733,382.283861,1.112121,0.500019,0.0,0.500014,0.0,...,0.374259,0.369403,0.369585,0.373192,0.378285,0.371398,0.424041,0.432245,0.439441,0.435905
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,32.0,0.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,131.000000,2.000000,10.000000,10.000000,1.000000,0.000000,32.0,0.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,257.000000,4.000000,19.000000,19.000000,2.000000,0.000000,32.0,1.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,384.000000,6.000000,39.000000,39.000000,3.000000,1.000000,32.0,1.000000,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,64.000000,512.000000,7.000000,9170.000000,9904.000000,4.000000,1.000000,32.0,1.000000,2.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [71]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))

In [40]:
dfConv.to_csv(os.path.join(SAVE_DIR,'Data_convolution_%s.csv'%model_name), index = False)